Use with `vscode` and its `Data Wrangler`+`Jupyter` extension. Check in `JUPYTER` Bottom Tab.

*known bug:* if **keyname** has special char, update into globals **varname** space will crash vscode, which cause memory and CPU usage get abnormally higher.

**YOU MUST RESTART VSCODE** after editing source code when that crash happens.

In [1]:
import os
import sys
import numpy as np
try:
    _DIR_CURRENT = os.path.dirname(os.path.abspath(__file__))
except NameError:
    _DIR_CURRENT = os.getcwd()
_DIR_ROOT = os.path.dirname(_DIR_CURRENT)
sys.path.insert(0, _DIR_ROOT)
from src.mocap_wrapper.script.data_viewer import convert_npt

In [2]:
_GLOBAL = convert_npt(files=[
    # '/home/n/document/code/mocap/output/wave_hands/mocap_wave_hands.npz',
    '/home/n/document/code/mocap/output/finger_dance/hmr4d_results_0.pt',
    # '/home/n/document/code/mocap/output/my_hand/mocap_my_hand.npz',
    # '/home/n/document/code/mocap/output/finger_dance/mocap_finger_dance.npz',
    # '/home/n/document/code/mocap/output/finger_dance/preprocess/slam_results.pt',
    # '/home/n/document/code/mocap/output/finger_dance/preprocess/vit_features_0.pt',
    # '/home/n/document/code/mocap/output/finger_dance/preprocess/vitpose_0.pt',
],
save=False,
Print=True,
)
globals().update(_GLOBAL)

load as dict from /home/n/document/code/mocap/output/finger_dance/hmr4d_results_0.pt
{
  "smpl_params_global一body_pose": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      21,
      3
    ]
  },
  "smpl_params_global一betas": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      10
    ]
  },
  "smpl_params_global一global_orient": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      3
    ]
  },
  "smpl_params_global一transl": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      3
    ]
  },
  "smpl_params_incam一global_orient": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      3
    ]
  },
  "smpl_params_incam一transl": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape": [
      149,
      3
    ]
  },
  "K_fullimg": {
    "type": "torch.Tensor",
    "dtype": "torch.dtype",
    "shape"

In [3]:
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import Dropdown, IntSlider, VBox, HBox, Output, Checkbox
import ipywidgets as widgets

def create_interactive_plot(data, title=""):
    """
    创建交互式多维数据可视化
    
    Parameters:
    data: numpy array, 任意形状的数据
    title: str, 图表标题
    """
    if not isinstance(data, np.ndarray):
        data = np.array(data)
    
    shape = data.shape
    ndim = len(shape)
    
    # 创建输出容器
    output = Output()
    
    # Y轴维度选择下拉框
    y_axis_dropdown = Dropdown(
        options=[(f'维度 {i} (size: {shape[i]})', i) for i in range(ndim)],
        value=0,
        description='Y轴:',
        style={'description_width': 'initial'}
    )
    
    # 滑块容器
    sliders_container = VBox([])
    
    # 为其他维度创建索引选择滑块和复选框
    index_sliders = {}
    show_all_checkboxes = {}
    
    def create_sliders_for_dimension(y_dim):
        """为指定的Y轴维度创建滑块和复选框"""
        # 清除现有控件
        for slider in index_sliders.values():
            slider.close()
        for checkbox in show_all_checkboxes.values():
            checkbox.close()
        index_sliders.clear()
        show_all_checkboxes.clear()
        
        # 创建新的控件（除了Y轴维度）
        control_widgets = []
        for i in range(ndim):
            if i != y_dim:
                # 创建滑块
                slider = IntSlider(
                    value=0,
                    min=0,
                    max=shape[i]-1,
                    step=1,
                    description=f'索引:',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='300px')
                )
                
                # 创建复选框
                checkbox = Checkbox(
                    value=True,
                    description=f'固定维度{i}',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='100px')
                )
                
                # 绑定事件
                slider.observe(update_plot, names='value')
                checkbox.observe(update_plot, names='value')
                
                # 存储引用
                index_sliders[i] = slider
                show_all_checkboxes[i] = checkbox
                
                # 创建水平布局：滑块 + 复选框
                control_row = HBox([checkbox,slider])
                control_widgets.append(control_row)
        
        # 更新滑块容器
        sliders_container.children = control_widgets
    
    def update_sliders(change=None):
        """当Y轴维度改变时，更新滑块"""
        y_dim = y_axis_dropdown.value
        create_sliders_for_dimension(y_dim)
        update_plot()
    
    def update_plot(change=None):
        """更新图表"""
        with output:
            output.clear_output(wait=True)
            
            y_dim = y_axis_dropdown.value
            
            # 创建图表
            fig = go.Figure()
            
            # 获取需要显示全部数据的维度
            show_all_dims = []
            fixed_indices = {}
            
            for dim, checkbox in show_all_checkboxes.items():
                if not checkbox.value:
                    show_all_dims.append(dim)
                else:
                    fixed_indices[dim] = index_sliders[dim].value
            
            try:
                if not show_all_dims:
                    # 没有维度需要显示全部，按原来的方式处理
                    indices = [slice(None)] * ndim
                    indices[y_dim] = slice(None)
                    
                    for dim, value in fixed_indices.items():
                        indices[dim] = value
                    
                    filtered_data = data[tuple(indices)]
                    
                    if filtered_data.ndim > 1:
                        filtered_data = filtered_data.flatten()
                    
                    x_values = np.arange(len(filtered_data))
                    
                    fig.add_trace(go.Scatter(
                        x=x_values,
                        y=filtered_data,
                        mode='lines+markers',
                        name='数据',
                        line=dict(width=2),
                        marker=dict(size=4)
                    ))
                    
                    title_info = f""
                    if fixed_indices:
                        index_info = [f"维度{dim}[{value}]" for dim, value in fixed_indices.items()]
                        title_info += f"固定: {', '.join(index_info)}"
                
                else:
                    # 有维度需要显示全部
                    # 生成颜色
                    colors = px.colors.qualitative.Set1
                    color_idx = 0
                    
                    # 计算需要绘制的线条数量
                    total_lines = 1
                    for dim in show_all_dims:
                        total_lines *= shape[dim]
                    
                    # 限制最大线条数量以避免性能问题
                    if total_lines > 50:
                        print(f"警告: 需要绘制 {total_lines} 条线，可能会影响性能")
                    
                    # 生成所有需要显示的索引组合
                    from itertools import product
                    
                    show_all_ranges = [range(shape[dim]) for dim in show_all_dims]
                    
                    for combo in product(*show_all_ranges):
                        # 构建索引
                        indices = [slice(None)] * ndim
                        indices[y_dim] = slice(None)
                        
                        # 设置固定维度的索引
                        for dim, value in fixed_indices.items():
                            indices[dim] = value
                        
                        # 设置显示全部维度的当前索引
                        for i, dim in enumerate(show_all_dims):
                            indices[dim] = combo[i]
                        
                        filtered_data = data[tuple(indices)]
                        
                        if filtered_data.ndim > 1:
                            filtered_data = filtered_data.flatten()
                        
                        x_values = np.arange(len(filtered_data))
                        
                        # 生成线条标签
                        combo_labels = [f"维度{dim}[{combo[i]}]" for i, dim in enumerate(show_all_dims)]
                        line_name = ', '.join(combo_labels)
                        
                        fig.add_trace(go.Scatter(
                            x=x_values,
                            y=filtered_data,
                            mode='lines+markers',
                            name=line_name,
                            line=dict(
                                width=2,
                                color=colors[color_idx % len(colors)]
                            ),
                            marker=dict(size=3)
                        ))
                        
                        color_idx += 1
                    
                    title_info = []
                    # if show_all_dims:
                    #     show_all_info = [f"维度{dim}" for dim in show_all_dims]
                    #     title_info.append(f"显示全部: {', '.join(show_all_info)}")
                    if fixed_indices:
                        fixed_info = [f"维度{dim}[{value}]" for dim, value in fixed_indices.items()]
                        title_info.append(f"\t固定: {', '.join(fixed_info)}")
                    title_info = ', '.join(title_info)
                
                fig.update_layout(
                    title=f"{title}<br><sub>{title_info}</sub>",
                    xaxis_title="索引",
                    yaxis_title=f"维度{y_dim}的值",
                    hovermode='x unified',
                    template='plotly_white',
                    height=600,
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=1.01
                    )
                )
                
                # 添加数据信息
                info_text = f"数据形状: {shape}"
                if show_all_dims:
                    info_text += f"<br>显示全部维度: {show_all_dims}"
                
                fig.add_annotation(
                    text=info_text,
                    xref="paper", yref="paper",
                    x=0.02, y=0.98,
                    showarrow=False,
                    bgcolor="rgba(255,255,255,0.8)",
                    bordercolor="gray",
                    borderwidth=1
                )
                
                fig.show()
                
            except Exception as e:
                print(f"绘图错误: {e}")
                import traceback
                traceback.print_exc()
    
    # 初始化
    create_sliders_for_dimension(y_axis_dropdown.value)
    
    # 绑定事件
    y_axis_dropdown.observe(update_sliders, names='value')
    
    # 创建控制面板
    controls_box = VBox([
        y_axis_dropdown,
        sliders_container
    ])
    
    # 显示界面
    display(controls_box)
    display(output)
    update_plot()

In [ ]:
# use `ctrl+space` to auto-complete keyname
_datas = [
    _GLOBAL['smplx_wilor_hand0R_0_hand_pose'],
    _GLOBAL['smplx_wilor_hand0R_0_global_orient'],
]
for d in _datas:
    # d = d*180/np.pi
    create_interactive_plot(d)

Output()

Output()

Output()

In [ ]:
# from src.mocap_wrapper.run.lib import savez
# savez('/home/n/document/code/mocap/output/wave_hands/mocap.npz', {
#     'smplx;wilor;hand1R;0;global_orient':_GLOBAL['smplx_wilor_hand0R_0_global_orient'],
# })